In [1]:
from datasets import load_dataset
from random import randrange

# Load dataset from the hub
dataset = load_dataset("/mnt/d/datasets/huggingface/spider/", split="train")

print(f"dataset size: {len(dataset)}")
print(dataset[randrange(len(dataset))])
# dataset size: 7000

/home/camembert/miniconda3/envs/anything/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


dataset size: 7000
{'db_id': 'program_share', 'query': 'SELECT t1.name FROM program AS t1 JOIN broadcast AS t2 ON t1.program_id  =  t2.program_id GROUP BY t2.program_id ORDER BY count(*) DESC LIMIT 1', 'question': 'Which program is broadcast most frequently? Give me the program name.', 'query_toks': ['SELECT', 't1.name', 'FROM', 'program', 'AS', 't1', 'JOIN', 'broadcast', 'AS', 't2', 'ON', 't1.program_id', '=', 't2.program_id', 'GROUP', 'BY', 't2.program_id', 'ORDER', 'BY', 'count', '(', '*', ')', 'DESC', 'LIMIT', '1'], 'query_toks_no_value': ['select', 't1', '.', 'name', 'from', 'program', 'as', 't1', 'join', 'broadcast', 'as', 't2', 'on', 't1', '.', 'program_id', '=', 't2', '.', 'program_id', 'group', 'by', 't2', '.', 'program_id', 'order', 'by', 'count', '(', '*', ')', 'desc', 'limit', 'value'], 'question_toks': ['Which', 'program', 'is', 'broadcast', 'most', 'frequently', '?', 'Give', 'me', 'the', 'program', 'name', '.']}


In [2]:
# import pandas as pd
# df = pd.read_json("data/spider_train.json")

In [3]:
dataset1 = load_dataset('json', data_files="data/spider_train.json")
                                              

In [4]:
dataset

Dataset({
    features: ['db_id', 'query', 'question', 'query_toks', 'query_toks_no_value', 'question_toks'],
    num_rows: 7000
})

In [5]:
dataset1['train']

Dataset({
    features: ['history', 'db_id', 'instruction', 'output', 'input'],
    num_rows: 8659
})

In [6]:
print(dataset1['train'][0]['instruction'])

I want you to act as a SQL terminal in front of an example database, you need only to return the sql command to me.Below is an instruction that describes a task, Write a response that appropriately completes the request.
"
##Instruction:
department_management contains tables such as department, head, management. Table department has columns such as Department_ID, Name, Creation, Ranking, Budget_in_Billions, Num_Employees. Department_ID is the primary key.
Table head has columns such as head_ID, name, born_state, age. head_ID is the primary key.
Table management has columns such as department_ID, head_ID, temporary_acting. department_ID is the primary key.
The head_ID of management is the foreign key of head_ID of head.
The department_ID of management is the foreign key of Department_ID of department.




In [7]:
def format_instruction(sample):
    return f"""{sample['instruction']}{sample['input']}
{sample['output']}
"""

In [8]:
print(format_instruction(dataset1['train'][0]))

I want you to act as a SQL terminal in front of an example database, you need only to return the sql command to me.Below is an instruction that describes a task, Write a response that appropriately completes the request.
"
##Instruction:
department_management contains tables such as department, head, management. Table department has columns such as Department_ID, Name, Creation, Ranking, Budget_in_Billions, Num_Employees. Department_ID is the primary key.
Table head has columns such as head_ID, name, born_state, age. head_ID is the primary key.
Table management has columns such as department_ID, head_ID, temporary_acting. department_ID is the primary key.
The head_ID of management is the foreign key of head_ID of head.
The department_ID of management is the foreign key of Department_ID of department.

###Input:
How many heads of the departments are older than 56 ?

###Response:
SELECT count(*) FROM head WHERE age  >  56



In [9]:
# def format_instruction(sample):
# 	return f"""### Instruction:
# Use the Input below to create an instruction, which could have been used to generate the input using an LLM.

# ### Input:
# {sample['response']}

# ### Response:
# {sample['instruction']}
# """


In [10]:
# ### Instruction:
# Use the Input below to create an instruction, which could have been used to generate the input using an LLM.

# ### Input:
# 22nd July 1947

# ### Response:
# When was the Indian National Flag adopted


In [11]:
# { 
#         "db_id" : "department_management" , 
#         "instruction" : "예제 데이터베이스 앞에서 SQL 터미널 역할을 하길 원합니다. sql 명령을 나에게 반환하기만 하면 됩니다. 다음은 작업을 설명하는 지침입니다. 요청을 적절하게 완료하는 응답을 작성하십시오. \n \" \n ##지침: \n Department_management에는 부서, 책임자, 관리와 같은 테이블이 포함되어 있습니다. 테이블 부서에는 Department_ID, 이름, 생성, 순위, Budget_in_Billions, Num_Employees와 같은 열이 있습니다. Department_ID가 기본 키입니다. \n 테이블 head에는 head_ID, 이름, Born_state, age와 같은 열이 있습니다. head_ID는 기본 키입니다. \n 테이블 관리에는 Department_ID, head_ID, temporary_acting과 같은 열이 있습니다. Department_ID가 기본 키입니다. \n 관리의 head_ID는 head의 head_ID의 외래 키입니다. \n 관리 부서_ID는 부서 부서_ID의 외래 키입니다. \n \n " , 
#         "input" : "###Input: \n 56세 이상인 부서장은 몇 명입니까? \n \n ###응답:" , 
#         "output" : "SELECT count(*) FROM head WHERE age > 56" , 
#         "history" : [] 
#     },

In [12]:
# from random import randrange

# print(format_instruction(dataset[randrange(len(dataset))]))


In [13]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

use_flash_attention = True

# Hugging Face model id
model_id = "/mnt/d/models/huggingface/CodeLlama-7b-hf"  # non-gated
# model_id = "meta-llama/Llama-2-7b-hf" # gated


# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    use_cache=False,
    # attn_implementation="flash_attention_2",
    use_flash_attention_2=use_flash_attention,
    device_map="auto",
)
model.config.pretraining_tp = 1


tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"


[2024-01-28 23:51:11,384] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:48<00:00, 24.13s/it]


In [14]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

# LoRA config based on QLoRA paper
peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.1,
        r=64,
        bias="none",
        task_type="CAUSAL_LM",
)


# prepare model for training
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)


In [23]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="CodeLlama-7b-spider",
    num_train_epochs=3,
    per_device_train_batch_size=6 if use_flash_attention else 4,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=2e-4,
    bf16=True,
    tf32=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    report_to="none",
    # disable_tqdm=True # disable tqdm since with packing values are in correct
)


In [24]:
dataset1['train'][0]

{'history': [],
 'db_id': 'department_management',
 'instruction': 'I want you to act as a SQL terminal in front of an example database, you need only to return the sql command to me.Below is an instruction that describes a task, Write a response that appropriately completes the request.\n"\n##Instruction:\ndepartment_management contains tables such as department, head, management. Table department has columns such as Department_ID, Name, Creation, Ranking, Budget_in_Billions, Num_Employees. Department_ID is the primary key.\nTable head has columns such as head_ID, name, born_state, age. head_ID is the primary key.\nTable management has columns such as department_ID, head_ID, temporary_acting. department_ID is the primary key.\nThe head_ID of management is the foreign key of head_ID of head.\nThe department_ID of management is the foreign key of Department_ID of department.\n\n',
 'output': 'SELECT count(*) FROM head WHERE age  >  56',
 'input': '###Input:\nHow many heads of the depart

In [25]:
from trl import SFTTrainer

max_seq_length = 2048 # max sequence length for model and packing of the dataset

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset1['train'],
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,
    formatting_func=format_instruction,
    args=args,
)


Generating train split: 2203 examples [00:01, 1140.84 examples/s]


In [26]:
# train
trainer.train() # there will not be a progress bar since tqdm is disabled




Step,Training Loss
10,0.438400
20,0.346200
30,0.260900
40,0.231600
50,0.208700
60,0.197500
70,0.186400
80,0.186500
90,0.185000
100,0.164400


/home/camembert/miniconda3/envs/anything/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/camembert/miniconda3/envs/anything/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


TrainOutput(global_step=552, training_loss=0.13239662856727408, metrics={'train_runtime': 15848.0639, 'train_samples_per_second': 0.417, 'train_steps_per_second': 0.035, 'total_flos': 5.393218968779489e+17, 'train_loss': 0.13239662856727408, 'epoch': 3.0})

In [27]:
# save model
trainer.save_model()